# Linear Regression with PySpark

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("crew_prediction").getOrCreate()

In [ ]:
df = spark.read.csv("../input/cruise-used-for-pyspark/cruise_dataset.csv", inferSchema=True, header=True)

In [ ]:
df.printSchema()

In [ ]:
df.show()

In [ ]:
df.describe().show()

In [ ]:
df.groupBy("Cruise_line").count().show()

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruise_cat")
indexed = indexer.fit(df).transform(df)

In [ ]:
indexed.show()

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
indexed.columns

In [ ]:
assembler = VectorAssembler(
    inputCols=['Age',
               'Tonnage',
               'passengers',
               'length',
               'cabins',
               'passenger_density',
               'cruise_cat'],
    outputCol="features"
)

In [ ]:
output = assembler.transform(indexed)

In [ ]:
output.select("features","crew").show()

In [ ]:
final_data = output.select("features","crew")

In [ ]:
train_data, test_data = final_data.randomSplit([0.7, 0.3], seed=42)

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
lr = LinearRegression(labelCol="crew")

In [ ]:
model = lr.fit(train_data)

In [ ]:
import pandas as pd

In [ ]:
coeff = model.coefficients
col_names = ['Age',
               'Tonnage',
               'passengers',
               'length',
               'cabins',
               'passenger_density',
               'cruise_cat']

In [ ]:
pd.DataFrame({"Variable Names":col_names, "Coefficients":coeff})

In [ ]:
res = model.evaluate(test_data)

In [ ]:
print(f"""
Linear Regression Results Report
==================================
RMSE:\t {res.rootMeanSquaredError}
MSE:\t {res.meanSquaredError}
R2:\t {res.r2}
Adj R2:\t{res.r2adj}
==================================
"""
)

In [ ]:
from pyspark.sql.functions import corr

In [ ]:
df.select(corr('crew', 'passengers')).show()

In [ ]:
df.select(corr('crew', 'cabins')).show()

In [ ]:
df.select(corr('crew', 'Age')).show()